In [7]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.action_chains import ActionChains
from selenium.common.exceptions import NoSuchElementException
from selenium.webdriver.chrome.options import Options
from selenium.common.exceptions import TimeoutException
from selenium.common.exceptions import StaleElementReferenceException
import os
from bs4 import BeautifulSoup
import matplotlib.pyplot as plt
import numpy as np 
import glob
import time
import yfinance as yf
import datetime
import requests
import shutil


ERROR = False

cookies_ec_dict = {'google': EC.element_to_be_clickable((By.XPATH, "//div[@class='QS5gu sy4vM']")),
                'nasdaq': EC.element_to_be_clickable((By.ID, "onetrust-accept-btn-handler")),
                'investing.com':EC.element_to_be_clickable((By.ID, "onetrust-accept-btn-handler")),
                'zacks.com_readmore': EC.visibility_of_element_located((By.XPATH, "//button[contains(@class, 'Button__StyledButton') and contains(text(), 'Read more to accept preferences')]")),
                'zacks.com': EC.element_to_be_clickable((By.XPATH, "//button[contains(@class, 'Button__StyledButton-a1qza5-0') and contains(text(), 'Accept all')]")),
                'tipranks.com': EC.element_to_be_clickable((By.CLASS_NAME, "Button__StyledButton-a1qza5-0"))}

nav_nasdaq_ec_dict = {'select_date': EC.element_to_be_clickable((By.ID, "date-picker__toggle")),
                            #"//button[@class='date-picker__toggle' and @aria-label='click show datepicker popup']")),      
                      'select_searchbox':  EC.visibility_of_element_located((By.CLASS_NAME, "date-picker__input")),
                      'apply_date_button': EC.element_to_be_clickable((By.XPATH,
                            "//button[@class='date-picker__apply']"))}

nav_investingcom_ec_dict = {'select_calendar': EC.element_to_be_clickable((By.ID, "datePickerToggleBtn"))}

buysell_tipranks_dict = {'Strong Buy': '1', 'Moderate Buy': '2', 'Hold': '3', }

def accept_cookies(site = 'google'):   
    if site == 'zacks.com':
        try:
            read_more = WebDriverWait(driver, 5).until(cookies_ec_dict['zacks.com_readmore']) 
            read_more.click()
        except:
            print("No 'Read more' button found on ", site)
        try:
            accept = WebDriverWait(driver, 5).until(cookies_ec_dict[site])
            accept.click()
        except:
            print("No 'Accept' button found or unable to click on ", site)
    
    elif site == 'tipranks.com':
        try:
            accept_button = WebDriverWait(driver, 5).until(cookies_ec_dict['tipranks.com'])
            accept_button.click()
            print("Accept button clicked successfully")
        except Exception as e:
            print("Error:", e)

    elif site == 'stockinvest.us':
        pass

    elif site == 'tradingview.com':
        pass

    elif site == 'investing.com':
        accept = WebDriverWait(driver, 5).until(cookies_ec_dict[site])
        accept.click()



def google(query, cookies):
    # Open the website
    driver.get("https://google.com")
    if cookies: accept_cookies('google')
    search_box = driver.find_element(By.NAME, "q")

     # Enter your search query
    search_query = query
    search_box.send_keys(search_query)

    # Simulate hitting Enter
    search_box.send_keys(Keys.RETURN)

def click_google_site(site):
    driver.find_element(By.XPATH, f"//a[contains(@href, '{site}')]").click()

def select_date(year, month, day, site = 'nasdaq'):
    if site == 'nasdaq':
        date = datetime.date(year, month, day).strftime('%m/%d/%Y')
        WebDriverWait(driver, 1).until(nav_nasdaq_ec_dict['select_date']).click()
        driver.execute_script(f"document.querySelector('.date-picker__input').value = '{date}';")
        WebDriverWait(driver, 1).until(nav_nasdaq_ec_dict['e_button']).click()
    elif site == 'investing.com':
        date = datetime.date(year, month, day).strftime('%m/%d/%Y')
        WebDriverWait(driver, 1).until(nav_investingcom_ec_dict['select_calendar']).click()
        input_element = driver.find_element(By.CSS_SELECTOR, "input.newInput#startDate")
        input_element.clear()  # Clear any existing value
        input_element.send_keys(date)  
        input_element = driver.find_element(By.CSS_SELECTOR, "input.newInput#endDate")
        input_element.clear()  # Clear any existing value
        input_element.send_keys(date)  
        driver.find_element(By.ID, 'applyBtn').click()
            
def google_site(query, site, cookies):
    google(query, cookies)
    click_google_site(site)


def get_symbols(site, openclose = False):
    time.sleep(1)

    symbols = []
    if openclose: timings = []
    if site == 'investing.com':
        openclose_dict = {'1': 'open', '2': 'none', '3': 'close'}
        table = driver.find_element(By.ID, "earningsCalendarData")
        tbody = table.find_element(By.TAG_NAME, "tbody")
        rows = tbody.find_elements(By.TAG_NAME, "tr")
        # Iterate over each table row
        for index, row in enumerate(rows):
            if index == 0: continue
            try:
                inner_html = row.get_attribute("innerHTML")
                soup = BeautifulSoup(inner_html, "html.parser")
                a_element = soup.find("a", class_="bold middle")
                if openclose:
                    timing_element = soup.find("td", class_="right time")
                    timing_value = timing_element.get("data-value")
                    timings.append(openclose_dict[timing_value])
                if a_element:
                    company = a_element.text.strip()
                    symbols.append(company)
            except Exception as e:
                print(f"Error processing row {index}: {e}")
                continue
    if openclose: return symbols, timings
    return symbols

def search_on(website, query):
    if website == 'zacks.com':
        search_input = WebDriverWait(driver, 20).until(
            EC.visibility_of_element_located((By.ID, "search-q"))
        )
        # Input the search query
        search_input.send_keys(query)
        # Press ENTER to submit the form
        search_input.send_keys(Keys.ENTER)


def get_rank(website):
    time.sleep(0.5)

    if website == 'zacks.com':
        try:
            # Wait for the paragraph element to be visible
            rank_paragraph = WebDriverWait(driver, 2).until(
                EC.visibility_of_element_located((By.CLASS_NAME, "rank_view"))
            )

            # Get the text of the paragraph element
            rank_text = rank_paragraph.text.strip()
            return int(rank_text[0])
        
        except Exception as e:
            ERROR = True


    elif website == 'tradingview.com':
        try:
            speedometer_div = WebDriverWait(driver, 10).until(
                EC.visibility_of_element_located((By.CSS_SELECTOR, ".speedometerWrapper-kg4MJrFB.summary-kg4MJrFB"))
            )
            # Get the outer HTML content of the div element
            html_content = speedometer_div.get_attribute("outerHTML")
            soup = BeautifulSoup(html_content, 'html.parser')
            counter_spans = soup.find_all('span', class_='counterNumber-kg4MJrFB')
            numbers = [int(span.text) for span in counter_spans]
            if numbers is not None and sum(numbers)!=0:
                return (-numbers[0]+numbers[-1])/sum(numbers)
        except Exception as e:
            print('falha')
            ERROR = True
            

    elif website == 'tipranks.com':
        try:
            rank_span = WebDriverWait(driver, 2).until(
                EC.visibility_of_element_located((By.CSS_SELECTOR, ".colorpale.fonth4_bold.aligncenter"))
            )

            # Get the text of the span element
            rank_text = rank_span.text.strip()
            return rank_text
        except Exception as e:
            ERROR = True

    elif website == 'stockinvest.us':
        try:
            technicals_rank = WebDriverWait(driver, 5).until(
                EC.visibility_of_element_located((By.XPATH, "//span[@class='pl-5 pr-5 pt-3 pb-3 font-weight-600 border-radius-default font-size-16']"))
            )

            # Get the text of the element
            technicals_rank = number_element.text.strip()
            print(technicals_rank)
            return technicals_rank      
        except Exception as e:
            print('falha')
            ERROR = True




            
def analyst(website, symbol, cookies):
    if website == 'zacks.com':
        driver.get(f'https://www.zacks.com/stock/quote/{symbol}?q={symbol}')
    elif website == 'tipranks.com':
        driver.get(f'https://www.tipranks.com/stocks/{symbol}/forecast')
    elif website == 'stockinvest.us':
        driver.get(f'https://stockinvest.us/stock/{symbol}')
    elif website == 'tradingview.com':
        driver.get(f'https://www.tradingview.com/symbols/{symbol}/technicals')
    elif website == 'stockanalysis.com':
        driver.get(f'https://www.stockanalysis.com/stocks/{symbol}/forecast/')
    else:
        google_site(f'{website[:website.find(".")]} {symbol}', website, cookies)
    if cookies: accept_cookies(website)
    rank = get_rank(website)
    return rank
   
def earnings_rank(website, year, month, day, path = ''):
    if path != '':
        if path[-1] == '/': path = path[:-1]
    if os.path.exists(path+'/'+f'{year}-{month}-{day}') is False:   
        print('path does not exist')
    else:
        for index, tickerpath in enumerate(glob.glob(path+'/'+f'{year}-{month}-{day}/*/')):
            ticker = tickerpath[len(path+'/'+f'{year}-{month}-{day}/'):-1]
            rank = analyst(website, ticker, cookies = not(index))
            print(ticker, rank)
            with open(tickerpath+website, 'a') as f:
                f.write(f'{datetime.datetime.today().year}-{datetime.datetime.today().month}-{datetime.datetime.today().day},    {rank}\n')


def get_tickers_for_earnings_date(year, month, day, openclose = False):
    #google_site('earnings calendar', 'investing.com', cookies = True)
    driver.get('https://www.investing.com/earnings-calendar/')
    time.sleep(0.5)
    accept_cookies('investing.com')
    time.sleep(0.5)
    select_date(year, month, day, site = 'investing.com')
    if openclose: 
        tickers, timings = get_symbols('investing.com', openclose = openclose)
        return tickers, timings
    else:
        tickers = get_symbols('investing.com', openclose = openclose)
        return tickers
 

def make_earnings_dates_directories(path, year, month, day, tickers, openclose  = None):
    if path[-1] == '/': path = path[:-1]
    if os.path.exists(path) is False:
        os.mkdir(path)
    if os.path.exists(path+'/'+f'{year}-{month}-{day}') is False:
        os.mkdir(path+'/'+f'{year}-{month}-{day}')  
    for index, ticker in enumerate(tickers):
        if os.path.exists(path+'/'+f'{year}-{month}-{day}/'+ ticker) is False:
            os.mkdir(path+'/'+f'{year}-{month}-{day}/'+ ticker)
        if openclose is not None:
            with open(path+'/'+f'{year}-{month}-{day}/'+ ticker + '/openclose.dat', 'a') as f:
                f.write(f'{openclose[index]} \n')

    
         
def filter_by_lastrank(path, year, month, day, website):
    if path != '':
        if path[-1] == '/': path = path[:-1]
    if os.path.exists(path+'/'+f'{year}-{month}-{day}') is False:   
        print('path does not exist')
    else:
        ranks = ['1','2','3','4','5','None']
        ticker_rank = {rank:[] for rank in ranks}
        if website == 'zacks.com':
            for _, tickerpath in enumerate(glob.glob(path+'/'+f'{year}-{month}-{day}/*/')):
                ticker = tickerpath[len(path+'/'+f'{year}-{month}-{day}/'):-1]
                if os.path.exists(tickerpath+website):
                    with open(tickerpath+website, 'r') as f:
                        text = f.read()
                        last = text.split(' ')[-1]
                        ticker_rank[last[:last.find('\n')]].append(ticker)


        elif website == 'tradingview.com':
            for _, tickerpath in enumerate(glob.glob(path+'/'+f'{year}-{month}-{day}/*/')):
                ticker = tickerpath[len(path+'/'+f'{year}-{month}-{day}/'):-1]
                if os.path.exists(tickerpath+website):
                    with open(tickerpath+website, 'r') as f:
                        text = f.read()
                        try:
                            last = float(text.split(' ')[-1])
                            if last > 0.5:
                                ticker_rank['1'].append(ticker)
                            elif last <= 0.5 and last > 0.25:
                                ticker_rank['2'].append(ticker)
                            elif last >= -0.25 and last <= 0.25:
                                ticker_rank['3'].append(ticker)
                            elif last >= -0.5 and last < -0.25:
                                ticker_rank['4'].append(ticker)
                            elif last < -0.5:
                                ticker_rank['5'].append(ticker)
                        except:
                            ticker_rank['None'].append(ticker)


                    
        return ticker_rank

     

def get_tickers_on_trading212():
    url = "https://live.trading212.com/api/v0/equity/metadata/instruments"

    headers = {"Authorization": f"{open('.apikey2', 'r').read()[:-1]}"}

    response = requests.get(url, headers=headers)

    data = response.json()

    trading212_tickers = []
    for instrument in data:
        trading212_tickers.append(instrument['shortName']) 

    return trading212_tickers

def update_reporting_tickers(year, month, day, new_tickers):
    
    directory = f'earnings/{year}-{month}-{day}/'
    companies_already_saved = []
    for company in glob.glob(directory + '*/'):
        tic = company[company.find(directory)+len(directory):]
        tic = tic[:tic.find('/')]
        if tic not in new_tickers:
            shutil.rmtree(company)
##########################################


In [2]:
#trading212_tickers = get_tickers_on_trading212()
trading212_tickers = list(np.load('trading212tickers.npy'))

In [8]:
driver = webdriver.Chrome()
year, month, day = 2024, 3, 19
tickers, timings = get_tickers_for_earnings_date(year, month, day, openclose = True)
driver.quit()

In [9]:
for tic, tim in zip(tickers, timings):
    print(tic, tim)
print(len(tickers))

ZTO close
AHCHY none
TME open
CNM none
XPEV open
HQY none
SDHC close
CAL open
ORLA none
EVV none
HUYA open
CHFHY none
DEC open
ABSI none
WULF close
TSHA close
HROW close
PGEN none
CTRN open
RLMD close
DSGN close
KGEI none
CLRB none
CRVS open
CMCM none
KRON none
ATRA none
DRIO none
AATC none
SNCE none
SOLO none
UAMY none
TBIO none
NROM none
JT none
WEBB none
GENN none
GADS none
GCFB none
39


In [10]:
   
directory = f'earnings/{year}-{month}-{day}/'
companies_already_saved = []
print('Saved but not reporting tomorrow')
for company in glob.glob(directory + '*/'):
    tic = company[company.find(directory)+len(directory):]
    tic = tic[:tic.find('/')]
    companies_already_saved.append(tic)
    if tic not in tickers:
        print(tic)

print('\n')
print('Not saved, but reporting tomorrow')
for tic in tickers:
    if tic not in companies_already_saved:
        print(tic)
        

Saved but not reporting tomorrow


Not saved, but reporting tomorrow
ZTO
AHCHY
TME
CNM
XPEV
HQY
SDHC
CAL
ORLA
EVV
HUYA
CHFHY
DEC
ABSI
WULF
TSHA
HROW
PGEN
CTRN
RLMD
DSGN
KGEI
CLRB
CRVS
CMCM
KRON
ATRA
DRIO
AATC
SNCE
SOLO
UAMY
TBIO
NROM
JT
WEBB
GENN
GADS
GCFB


In [11]:
update_reporting_tickers(year, month, day, tickers)

In [12]:
make_earnings_dates_directories('earnings', year, month, day, tickers, openclose = timings)


In [13]:
year, month, day = 2024, 3, 19
driver = webdriver.Chrome()
#earnings_rank('zacks.com', year, month, day, path = 'earnings')
earnings_rank('tradingview.com', year, month, day, path = 'earnings')
driver.quit()

GCFB None
KGEI 5
ATRA 3
HROW None
XPEV 3
CNM 3
WULF 3
TSHA None
DSGN None
CRVS 3
UAMY None
JT None
CHFHY None
SNCE None
AATC None
CLRB 3
KRON 4
ORLA 4
WEBB None
CAL 4
DRIO 3
HUYA 3
RLMD 3
HQY 3
SOLO 3
AHCHY 4
CMCM None
EVV None
SDHC 3
DEC 3
TBIO 3
ZTO 4
ABSI 4
NROM None
CTRN None
GENN None
GADS None
TME 3
PGEN 3


In [14]:
driver = webdriver.Chrome()
earnings_rank('tradingview.com', year, month, day, path = 'earnings')
driver.quit()

GCFB 0.038461538461538464
KGEI 0.46153846153846156
ATRA -0.4230769230769231
HROW -0.5
XPEV 0.23076923076923078
CNM 0.5384615384615384
WULF -0.38461538461538464
TSHA 0.038461538461538464
DSGN -0.5384615384615384
CRVS -0.11538461538461539
UAMY 0.15384615384615385
falha
JT None
CHFHY 0.2727272727272727
falha
SNCE None
AATC -0.2692307692307692
CLRB -0.23076923076923078
KRON 0.11538461538461539
ORLA 0.3076923076923077
falha
WEBB None
CAL 0.38461538461538464
DRIO -0.4230769230769231
HUYA 0.5384615384615384
RLMD 0.07692307692307693
HQY 0.5
SOLO -0.5
AHCHY 0.3076923076923077
CMCM 0.23076923076923078
EVV 0.038461538461538464
SDHC 0.23529411764705882
DEC -0.38461538461538464
TBIO 0.2692307692307692
ZTO 0.4230769230769231
ABSI -0.15384615384615385
NROM 0.4230769230769231
CTRN -0.15384615384615385
falha
GENN None
GADS -0.5
TME 0.4230769230769231
PGEN -0.23076923076923078


In [15]:
year, month, day = 2024, 3, 19
ticker_rank_tradingview = filter_by_lastrank('earnings', year, month, day, 'tradingview.com')
ticker_rank_zacks = filter_by_lastrank('earnings', year, month, day, 'zacks.com')

up_to_ranking = 2

In [16]:
## zacks
total_number_of_companies = 0
zacks_companies = []
for i in range(1,up_to_ranking + 1):
    ticker_rank_on212 = [x for x in ticker_rank_zacks[f'{i}'] if x in trading212_tickers]
    zacks_companies.extend(ticker_rank_on212)
    print(f'{i}: ', ticker_rank_on212, '\n')
    total_number_of_companies += len(ticker_rank_on212)

with open(f'earnings/{year}-{month}-{day}/_zacks.txt', 'w+') as f:
    for company in zacks_companies:
        f.write(f'{company} \n')


print('total number of companies: ', total_number_of_companies)

1:  [] 

2:  [] 

total number of companies:  0


In [17]:
## tradingview
total_number_of_companies = 0
tradingview_companies = []
for i in range(1,up_to_ranking + 1):
    ticker_rank_on212 = [x for x in ticker_rank_tradingview[f'{i}'] if x in trading212_tickers]
    tradingview_companies.extend(ticker_rank_on212)
    print(f'{i}: ', ticker_rank_on212, '\n')
    total_number_of_companies += len(ticker_rank_on212)

with open(f'earnings/{year}-{month}-{day}/_tradingview.txt', 'w+') as f:
    for company in tradingview_companies:
        f.write(f'{company} \n')
    f.write(f'{len(tradingview_companies)}')


print('total number of companies: ', total_number_of_companies)

1:  ['HUYA'] 

2:  ['KGEI', 'CAL', 'TBIO', 'ZTO', 'TME'] 

total number of companies:  6


In [18]:
intersection_companies = list(set(zacks_companies) & set(tradingview_companies))
with open(f'earnings/{year}-{month}-{day}/_intersection.txt', 'w+') as f:
    for company in intersection_companies:
        f.write(f'{company} \n')
    f.write(f'{len(intersection_companies)}')
intersection_companies

[]

In [100]:
invested_initial_value = 1033.85
initial_value_per_ticker = invested_initial_value/len(zacks_companies)
total_initial_value = 0
total_final_value = 0
total_percentage = 0
start_date = str(datetime.date(year, month, day) - datetime.timedelta(days = 10))
if datetime.date(year, month, day).weekday() == 4:
    end_date = str(datetime.date(year, month, day) + datetime.timedelta(days = 4))
else:
    end_date = str(datetime.date(year, month, day) + datetime.timedelta(days = 2))

for tic in zacks_companies:
    yf_ticker = yf.download(tickers=tic, start=start_date, end =end_date, progress=False, ignore_tz=True)
    print('\n')
    print(tic)
    print(yf_ticker['Open'])
    open_values = yf_ticker['Open'].values
    initial_value = open_values[-3]
    final_value = open_values[-1]
    
    ## take some value when market is closed
    #yf_ticker = yf.download(tickers=tic, period = '1d', interval = '1m', progress=False, ignore_tz=True,  prepost= True)
    #final_value= yf_ticker['Open'].values[-1]
    
    percentage_difference = (final_value - initial_value)/initial_value*100
    print(f'{tic}, {np.round(initial_value, 2)},   {np.round(final_value, 2)},  \
            {np.round(initial_value_per_ticker*percentage_difference,2)},    \
            {np.round(initial_value_per_ticker + (initial_value_per_ticker*percentage_difference/100),2)}')
    total_initial_value += initial_value_per_ticker
    total_final_value += initial_value_per_ticker+(initial_value_per_ticker*percentage_difference/100)

# compare with spy
yf_ticker = yf.download(tickers='SPY', start=start_date, end =end_date, progress=False, ignore_tz=True)
open_values = yf_ticker['Open'].values
final_value = open_values[-1]
percentage_difference_spy = (open_values[-1] - open_values[-3])/open_values[-3]*100

# compare with nasdaq
yf_ticker = yf.download(tickers='NDAQ', start=start_date, end =end_date, progress=False, ignore_tz=True)
open_values = yf_ticker['Open'].values
final_value = open_values[-1]
percentage_difference_ndaq = (open_values[-1] - open_values[-3])/open_values[-3]*100


print('\n', total_initial_value, np.round(total_final_value,2), \
      np.round((total_final_value - total_initial_value)/total_initial_value*100,2), \
      np.round(percentage_difference_spy,2), np.round(percentage_difference_ndaq,2))

print(end_date)



STNE
Date
2024-03-08    16.650000
2024-03-11    16.730000
2024-03-12    16.959999
2024-03-13    17.180000
2024-03-14    17.250000
Name: Open, dtype: float64
STNE, 16.96,   17.25,              589.26,                350.51


CWCO
Date
2024-03-08    29.84
2024-03-11    29.65
2024-03-12    29.00
2024-03-13    28.26
2024-03-14    28.25
Name: Open, dtype: float64
CWCO, 29.0,   28.25,              -891.25,                335.7


XGN
Date
2024-03-08    1.91
2024-03-11    1.89
2024-03-12    1.85
2024-03-13    1.83
2024-03-14    1.84
Name: Open, dtype: float64
XGN, 1.85,   1.84,              -186.28,                342.75

 1033.85 1028.97 -0.47 0.69 -0.83
2024-03-20


In [101]:
invested_initial_value = 1011.28
initial_value_per_ticker = invested_initial_value/len(tradingview_companies)
total_initial_value = 0
total_final_value = 0
total_percentage = 0
if datetime.date(year, month, day).weekday() == 4:
    end_date = str(datetime.date(year, month, day) + datetime.timedelta(days = 4))
else:
    end_date = str(datetime.date(year, month, day) + datetime.timedelta(days = 2))
for tic in tradingview_companies:
    yf_ticker = yf.download(tickers=tic, start=start_date, end =end_date, progress=False, ignore_tz=True)
    open_values = yf_ticker['Open'].values
    initial_value = open_values[-3]
    final_value = open_values[-1]
    print('\n')
    print(tic)
    print(yf_ticker['Open'])
    
    ## take some value when market is closed
    #yf_ticker = yf.download(tickers=tic, period = '1d', interval = '1m', progress=False, ignore_tz=True,  prepost= True)
    #final_value= yf_ticker['Open'].values[-1]
    
    percentage_difference = (final_value - initial_value)/initial_value*100
    print(f'{tic}, {np.round(initial_value, 2)},   {np.round(final_value, 2)},  \
            {np.round(initial_value_per_ticker*percentage_difference,2)},    \
            {np.round(initial_value_per_ticker + (initial_value_per_ticker*percentage_difference/100),2)}')
    total_initial_value += initial_value_per_ticker
    total_final_value += initial_value_per_ticker+(initial_value_per_ticker*percentage_difference/100)

# compare with spy
yf_ticker = yf.download(tickers='SPY', start=start_date, end =end_date, progress=False, ignore_tz=True)
open_values = yf_ticker['Open'].values
final_value = open_values[-1]
percentage_difference_spy = (open_values[-1] - open_values[-3])/open_values[-3]*100

# compare with nasdaq
yf_ticker = yf.download(tickers='NDAQ', start=start_date, end =end_date, progress=False, ignore_tz=True)
open_values = yf_ticker['Open'].values
final_value = open_values[-1]
percentage_difference_ndaq = (open_values[-1] - open_values[-3])/open_values[-3]*100


print('\n', total_initial_value, np.round(total_final_value,2), \
      np.round((total_final_value - total_initial_value)/total_initial_value*100,2), \
      np.round(percentage_difference_spy,2), np.round(percentage_difference_ndaq,2))

print(end_date)



STNE
Date
2024-03-08    16.650000
2024-03-11    16.730000
2024-03-12    16.959999
2024-03-13    17.180000
2024-03-14    17.250000
Name: Open, dtype: float64
STNE, 16.96,   17.25,              172.92,                102.86


ERJ
Date
2024-03-08    21.320000
2024-03-11    21.360001
2024-03-12    21.090000
2024-03-13    21.200001
2024-03-14    22.200001
Name: Open, dtype: float64
ERJ, 21.09,   22.2,              532.25,                106.45


GLBS
Date
2024-03-08    2.24
2024-03-11    2.20
2024-03-12    2.19
2024-03-13    2.30
2024-03-14    2.31
Name: Open, dtype: float64
GLBS, 2.19,   2.31,              554.13,                106.67


DLO
Date
2024-03-08    17.320000
2024-03-11    16.990000
2024-03-12    17.090000
2024-03-13    17.059999
2024-03-14    17.500000
Name: Open, dtype: float64
DLO, 17.09,   17.5,              242.61,                103.55


ABEO
Date
2024-03-08    8.39
2024-03-11    7.88
2024-03-12    7.72
2024-03-13    8.20
2024-03-14    8.18
Name: Open, dtype: float64
ABE

In [102]:
invested_initial_value = 1045.17 
initial_value_per_ticker = invested_initial_value/len(intersection_companies)
total_initial_value = 0
total_final_value = 0
total_percentage = 0
if datetime.date(year, month, day).weekday() == 4:
    end_date = str(datetime.date(year, month, day) + datetime.timedelta(days = 4))
else:
    end_date = str(datetime.date(year, month, day) + datetime.timedelta(days = 2))
for tic in intersection_companies:
    yf_ticker = yf.download(tickers=tic, start=start_date, end =end_date, progress=False, ignore_tz=True)
    print('\n')
    print(tic)
    print(yf_ticker['Open'])
    open_values = yf_ticker['Open'].values
    initial_value = open_values[-3]
    final_value = open_values[-1]
    
    ## take some value when market is closed
    #yf_ticker = yf.download(tickers=tic, period = '1d', interval = '1m', progress=False, ignore_tz=True,  prepost= True)
    #final_value= yf_ticker['Open'].values[-1]
    
    percentage_difference = (final_value - initial_value)/initial_value*100
    print(f'{tic}, {np.round(initial_value, 2)},   {np.round(final_value, 2)},  \
            {np.round(initial_value_per_ticker*percentage_difference,2)},    \
            {np.round(initial_value_per_ticker + (initial_value_per_ticker*percentage_difference/100),2)}')
    total_initial_value += initial_value_per_ticker
    total_final_value += initial_value_per_ticker+(initial_value_per_ticker*percentage_difference/100)

# compare with spy
yf_ticker = yf.download(tickers='SPY', start=start_date, end =end_date, progress=False, ignore_tz=True)
open_values = yf_ticker['Open'].values
final_value = open_values[-1]
percentage_difference_spy = (open_values[-1] - open_values[-3])/open_values[-3]*100

# compare with nasdaq
yf_ticker = yf.download(tickers='NDAQ', start=start_date, end =end_date, progress=False, ignore_tz=True)
open_values = yf_ticker['Open'].values
final_value = open_values[-1]
percentage_difference_ndaq = (open_values[-1] - open_values[-3])/open_values[-3]*100


print('\n', total_initial_value, np.round(total_final_value,2), \
      np.round((total_final_value - total_initial_value)/total_initial_value*100,2), \
      np.round(percentage_difference_spy,2), np.round(percentage_difference_ndaq,2))

print(end_date)



STNE
Date
2024-03-08    16.650000
2024-03-11    16.730000
2024-03-12    16.959999
2024-03-13    17.180000
2024-03-14    17.250000
Name: Open, dtype: float64
STNE, 16.96,   17.25,              1787.15,                1063.04

 1045.17 1063.04 1.71 0.69 -0.83
2024-03-20


In [24]:
## just tradingview

year_i, month_i, day_i = 2024, 3, 4
year_f, month_f, day_f = 2024, 3, 15
up_to_ranking = 2

initial_date = datetime.date(year_i, month_i, day_i)
final_date = datetime.date(year_f, month_f, day_f)
current_date = initial_date
stream_1 = [initial_date]
stream_2 = []
index = 0
while (current_date < final_date):
    if current_date.weekday() == 4:
        next_date = current_date + datetime.timedelta(days = 3)
    else:
        next_date = current_date + datetime.timedelta(days = 1)
    
    if index % 2:
        stream_1.append(next_date)
    else:
        stream_2.append(next_date)

    current_date = next_date

    index += 1
print(stream_1, stream_2, '\n')




total_initial_sum = 1000
total_final_sum = 0

for stream in (stream_1, stream_2):


    invested_initial_value = total_initial_sum/2

    for date in stream:
        year = date.year
        month = date.month
        day = date.day 


        ticker_rank_tradingview = filter_by_lastrank('earnings', year, month, day, 'tradingview.com')



        ## tradingview
        total_number_of_companies = 0
        tradingview_companies = []
        for i in range(1,up_to_ranking + 1):
            ticker_rank_on212 = [x for x in ticker_rank_tradingview[f'{i}'] if x in trading212_tickers]
            tradingview_companies.extend(ticker_rank_on212)
            total_number_of_companies += len(ticker_rank_on212)



        initial_value_per_ticker = invested_initial_value/len(tradingview_companies)
        total_initial_value = 0
        total_final_value = 0
        total_percentage = 0        
        if datetime.date(year, month, day).weekday() == 0:
            start_date = str(datetime.date(year, month, day) - datetime.timedelta(days = 3))
        else:
            start_date = str(datetime.date(year, month, day) - datetime.timedelta(days = 1))
        if datetime.date(year, month, day).weekday() == 4:
            end_date = str(datetime.date(year, month, day) + datetime.timedelta(days = 4))
        else:
            end_date = str(datetime.date(year, month, day) + datetime.timedelta(days = 2))
        print(start_date, date, end_date)
        for tic in tradingview_companies:
            yf_ticker = yf.download(tickers=tic, start=start_date, end=end_date, progress=False, ignore_tz=True)
            #print(tic)
            #print(yf_ticker['Open'][-3:], '\n')
            open_values = yf_ticker['Open'].values
            initial_value = open_values[-3]
            final_value = open_values[-1]
            
            ## take some value when market is closed
            #yf_ticker = yf.download(tickers=tic, period = '1d', interval = '1m', progress=False, ignore_tz=True,  prepost= True)
            #final_value= yf_ticker['Open'].values[-1]
            
            percentage_difference = (final_value - initial_value)/initial_value*100
            total_initial_value += initial_value_per_ticker
            total_final_value += initial_value_per_ticker+(initial_value_per_ticker*percentage_difference/100)

        # compare with spy
        yf_ticker = yf.download(tickers='SPY', start=start_date, end =end_date, progress=False, ignore_tz=True)
        open_values = yf_ticker['Open'].values
        final_value = open_values[-1]
        percentage_difference_spy = (open_values[-1] - open_values[-3])/open_values[-3]*100

        # compare with nasdaq
        yf_ticker = yf.download(tickers='NDAQ', start=start_date, end =end_date, progress=False, ignore_tz=True)
        open_values = yf_ticker['Open'].values
        final_value = open_values[-1]
        percentage_difference_ndaq = (open_values[-1] - open_values[-3])/open_values[-3]*100

        print(date, round(total_initial_value,2), round(total_final_value,2), round((total_final_value - total_initial_value)/total_initial_value*100,2))
        print('\n')
        invested_initial_value = total_final_value

    total_final_sum += total_final_value
    print(round(total_initial_sum/2,2), round(total_final_value,2), round((total_final_value - total_initial_sum/2)/(total_initial_sum/2)*100,2))
    print('\n\n')
print(round(total_initial_sum,2), round(total_final_sum,2), round((total_final_sum - total_initial_sum)/total_initial_sum*100,2))


[datetime.date(2024, 3, 4), datetime.date(2024, 3, 6), datetime.date(2024, 3, 8), datetime.date(2024, 3, 12), datetime.date(2024, 3, 14)] [datetime.date(2024, 3, 5), datetime.date(2024, 3, 7), datetime.date(2024, 3, 11), datetime.date(2024, 3, 13), datetime.date(2024, 3, 15)] 

2024-03-01 2024-03-04 2024-03-06
2024-03-04 500.0 558.26 11.65


2024-03-05 2024-03-06 2024-03-08
2024-03-06 558.26 592.39 6.11


2024-03-07 2024-03-08 2024-03-12
2024-03-08 592.39 615.45 3.89


2024-03-11 2024-03-12 2024-03-14
2024-03-12 615.45 591.59 -3.88


2024-03-13 2024-03-14 2024-03-16
2024-03-14 591.59 581.16 -1.76


500.0 581.16 16.23



2024-03-04 2024-03-05 2024-03-07
2024-03-05 500.0 505.64 1.13


2024-03-06 2024-03-07 2024-03-09
2024-03-07 505.64 517.05 2.26


2024-03-08 2024-03-11 2024-03-13
2024-03-11 517.05 505.06 -2.32


2024-03-12 2024-03-13 2024-03-15
2024-03-13 505.06 500.61 -0.88


2024-03-14 2024-03-15 2024-03-19
2024-03-15 500.61 490.35 -2.05


500.0 490.35 -1.93



1000 1071.52 7.15


In [23]:
## intersection

year_i, month_i, day_i = 2024, 3, 4
year_f, month_f, day_f = 2024, 3, 15
up_to_ranking = 2

initial_date = datetime.date(year_i, month_i, day_i)
final_date = datetime.date(year_f, month_f, day_f)
current_date = initial_date
stream_1 = [initial_date]
stream_2 = []
index = 0
while (current_date < final_date):
    if current_date.weekday() == 4:
        next_date = current_date + datetime.timedelta(days = 3)
    else:
        next_date = current_date + datetime.timedelta(days = 1)
    
    if index % 2:
        stream_1.append(next_date)
    else:
        stream_2.append(next_date)

    current_date = next_date

    index += 1
print(stream_1, stream_2, '\n')




total_initial_sum = 1000
total_final_sum = 0

for stream in (stream_1, stream_2):


    invested_initial_value = total_initial_sum/2

    for date in stream:
        year = date.year
        month = date.month
        day = date.day 



        ticker_rank_tradingview = filter_by_lastrank('earnings', year, month, day, 'tradingview.com')
        ticker_rank_zacks = filter_by_lastrank('earnings', year, month, day, 'zacks.com')
        up_to_ranking = 2

        total_number_of_companies = 0
        zacks_companies = []
        for i in range(1,up_to_ranking + 1):
            ticker_rank_on212 = [x for x in ticker_rank_zacks[f'{i}'] if x in trading212_tickers]
            zacks_companies.extend(ticker_rank_on212)
            total_number_of_companies += len(ticker_rank_on212)

        total_number_of_companies = 0
        tradingview_companies = []
        for i in range(1,up_to_ranking + 1):
            ticker_rank_on212 = [x for x in ticker_rank_tradingview[f'{i}'] if x in trading212_tickers]
            tradingview_companies.extend(ticker_rank_on212)
            total_number_of_companies += len(ticker_rank_on212)

        intersection_companies = list(set(zacks_companies) & set(tradingview_companies))


        initial_value_per_ticker = invested_initial_value/len(intersection_companies)
        total_initial_value = 0
        total_final_value = 0
        total_percentage = 0        
        if datetime.date(year, month, day).weekday() == 0:
            start_date = str(datetime.date(year, month, day) - datetime.timedelta(days = 3))
        else:
            start_date = str(datetime.date(year, month, day) - datetime.timedelta(days = 1))
        if datetime.date(year, month, day).weekday() == 4:
            end_date = str(datetime.date(year, month, day) + datetime.timedelta(days = 4))
        else:
            end_date = str(datetime.date(year, month, day) + datetime.timedelta(days = 2))
        print(start_date, date, end_date)
        for tic in intersection_companies:
            yf_ticker = yf.download(tickers=tic, start=start_date, end=end_date, progress=False, ignore_tz=True)
            #print(tic)
            #print(yf_ticker['Open'][-3:], '\n')
            open_values = yf_ticker['Open'].values
            initial_value = open_values[-3]
            final_value = open_values[-1]
            
            ## take some value when market is closed
            #yf_ticker = yf.download(tickers=tic, period = '1d', interval = '1m', progress=False, ignore_tz=True,  prepost= True)
            #final_value= yf_ticker['Open'].values[-1]
            
            percentage_difference = (final_value - initial_value)/initial_value*100
            total_initial_value += initial_value_per_ticker
            total_final_value += initial_value_per_ticker+(initial_value_per_ticker*percentage_difference/100)

        # compare with spy
        yf_ticker = yf.download(tickers='SPY', start=start_date, end =end_date, progress=False, ignore_tz=True)
        open_values = yf_ticker['Open'].values
        final_value = open_values[-1]
        percentage_difference_spy = (open_values[-1] - open_values[-3])/open_values[-3]*100

        # compare with nasdaq
        yf_ticker = yf.download(tickers='NDAQ', start=start_date, end =end_date, progress=False, ignore_tz=True)
        open_values = yf_ticker['Open'].values
        final_value = open_values[-1]
        percentage_difference_ndaq = (open_values[-1] - open_values[-3])/open_values[-3]*100

        print(date, round(total_initial_value,2), round(total_final_value,2), round((total_final_value - total_initial_value)/total_initial_value*100,2))
        print('\n')
        invested_initial_value = total_final_value

    total_final_sum += total_final_value
    print(round(total_initial_sum/2,2), round(total_final_value,2), round((total_final_value - total_initial_sum/2)/(total_initial_sum/2)*100,2))
    print('\n\n')
print(round(total_initial_sum,2), round(total_final_sum,2), round((total_final_sum - total_initial_sum)/total_initial_sum*100,2))


[datetime.date(2024, 3, 4), datetime.date(2024, 3, 6), datetime.date(2024, 3, 8), datetime.date(2024, 3, 12), datetime.date(2024, 3, 14)] [datetime.date(2024, 3, 5), datetime.date(2024, 3, 7), datetime.date(2024, 3, 11), datetime.date(2024, 3, 13), datetime.date(2024, 3, 15)] 

2024-03-01 2024-03-04 2024-03-06
2024-03-04 500.0 487.88 -2.42


2024-03-05 2024-03-06 2024-03-08
2024-03-06 487.88 553.98 13.55


2024-03-07 2024-03-08 2024-03-12
2024-03-08 553.98 541.09 -2.33


2024-03-11 2024-03-12 2024-03-14
2024-03-12 541.09 587.06 8.5


2024-03-13 2024-03-14 2024-03-16
2024-03-14 587.06 547.07 -6.81


500.0 547.07 9.41



2024-03-04 2024-03-05 2024-03-07
2024-03-05 500.0 522.58 4.52


2024-03-06 2024-03-07 2024-03-09
2024-03-07 522.58 533.92 2.17


2024-03-08 2024-03-11 2024-03-13
2024-03-11 533.92 511.44 -4.21


2024-03-12 2024-03-13 2024-03-15
2024-03-13 511.44 534.17 4.44


2024-03-14 2024-03-15 2024-03-19
2024-03-15 534.17 492.6 -7.78


500.0 492.6 -1.48



1000 1039.66 3.97
